#UFSM00993 - Aprendizado de Máquina Profundo T1
####João Vitor Bernardi - 202410054
####Guilherme Patrício Pimentel - 202412576
---
Implementação de um Multi-layer Perceptron (MLP) com uma camada oculta para a aproximação de funções binárias.
<br>
<br>

In [35]:
import numpy as np

2. Considere que apenas a camada oculta tem uma funcão de ativação (adote ReLU) ( ˜ i.e., a camada de saída tem
ativação identidade). Otimize a MLP com uma função de perda que penaliza o quadrado do erro de estimação.

In [36]:
def Relu(n):
    return np.maximum(0,n)

def calculoLoss(saidaFinal, saidaCorreta):

    return (saidaFinal - saidaCorreta)**2

3. Inicialize os coeficientes das matrizes de pesos Ω de tamanho N × M com os valores Ωi,j ∼ N (0,
p
2/N) (He
initialization). Inicialize os coeficientes dos vetores de viéses β com um valor constante pequeno (e.g., βk = 0,1).

In [37]:
def calculoParametro(pAntigo, taxaAprendizado, gradiente):

    return pAntigo - taxaAprendizado * gradiente

def heInitialization(n_entradas,n_saidas):

    desvioPadrao = np.sqrt(2 / n_entradas)
    saida = np.random.randn(n_entradas, n_saidas) * desvioPadrao
    return saida

4. Considere o método do gradiente descendente para otimização. Determine as derivadas da função de perda em ˜
função de cada parâmetro e as implemente manualmente. Identifique os gradientes textualmente.


In [38]:
def backWardPass(entradas, saidaCorreta, saidaFinal,pesosFinais, saidaOculta, saidaOcultaLinear):

    # 1. Gradiente do viés da camada de saída (∇b2)
    # É o erro da previsão, derivado da função de perda (erro quadrático).
    gradienteB2 = 2*(saidaFinal - saidaCorreta)

    # 2. Gradiente dos pesos da camada de saída (∇W2)
    # É o erro da previsão, multiplicado pela saída da camada oculta.
    # A transposição (.T) é para ajustar as dimensões da matriz.
    saidaOculta_T = saidaOculta.T
    gradienteW2 = saidaOculta_T @ gradienteB2

    # 3. Gradiente do viés da camada oculta (∇b1)
    # Erro propagado para a camada oculta.
    pesosFinais_T = pesosFinais.T
    derivada_relu = (saidaOcultaLinear > 0).astype(int)
    gradienteB1 = (gradienteB2 @ pesosFinais_T) * derivada_relu # Multiplicação elemento a elemento

    # 4. Gradiente dos pesos da camada oculta (∇W1)
    # Erro da camada oculta, multiplicado pela entrada original.
    entradas_T = entradas.T
    gradienteW1 = entradas_T @ gradienteB1

    return gradienteW2, gradienteB2, gradienteW1, gradienteB1

5. Avalie experimentalmente o impacto de retreinar a MLP. Por fim, determine uma seed para os experimentos
seguintes. Descreva os resultados.
<br>
5.a. Resultado do primeiro treino da MLP, que após 8000 épocas conseguiu prever de forma correta todas as entradas da XOR no teste final realizado


In [39]:
epocas = 8000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = np.random.rand(2, 2)
pesosFinais = np.random.rand(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.260730
Época: 1000, loss médio: 0.259135
Época: 1500, loss médio: 0.258011
Época: 2000, loss médio: 0.257218
Época: 2500, loss médio: 0.256688
Época: 3000, loss médio: 0.256315
Época: 3500, loss médio: 0.256041
Época: 4000, loss médio: 0.255843
Época: 4500, loss médio: 0.255694
Época: 5000, loss médio: 0.255581
Época: 5500, loss médio: 0.255493
Época: 6000, loss médio: 0.255423
Época: 6500, loss médio: 0.254089
Época: 7000, loss médio: 0.169656
Época: 7500, loss médio: 0.001387
Época: 8000, loss médio: 0.000000

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.0002)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.9997)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.9997)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.0002)


5.b. Resultado do segundo treino da MLP, dessa vez com outros pesos iniciais escolhidos aleatoriamente pelo método da HeInitialization. Conseguiu aproximar em 2000 épocas.

In [49]:
epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.151750
Época: 1000, loss médio: 0.103305
Época: 1500, loss médio: 0.007572
Época: 2000, loss médio: 0.000007

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.0033)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.9959)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.9992)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.0004)


5.c Determinando a seed 25102005 para o processo da HeInitialization, os resultados da rede tornam-se constantes e previsíveis, já que o valor dos parâmetros iniciais sempre será o mesmo

In [41]:
epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(25102005) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.134331
Época: 1000, loss médio: 0.127551
Época: 1500, loss médio: 0.127538
Época: 2000, loss médio: 0.127538

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.0000)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.4949)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.9899)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.4949)


6. Avalie experimentalmente o impacto do numero de épocas e o valor da taxa de aprendizado. Descreva os
resultados.

6.a seed = 80, épocas = 1000, taxa de aprendizado = 0.01

In [42]:
epocas = 1000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.151750
Época: 1000, loss médio: 0.103305

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.4757)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.5812)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.9906)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (-0.0427)


Com seed fixada no número 80, a Loss média do programa diminuiu ao longo de 1000 épocas, mas não o suficiente para gerar resultados mais precisos.

6.b seed = 80, épocas = 5000, taxa de aprendizado = 0.01

In [43]:
epocas = 5000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.151750
Época: 1000, loss médio: 0.103305
Época: 1500, loss médio: 0.007572
Época: 2000, loss médio: 0.000007
Época: 2500, loss médio: 0.000000
Época: 3000, loss médio: 0.000000
Época: 3500, loss médio: 0.000000
Época: 4000, loss médio: 0.000000
Época: 4500, loss médio: 0.000000
Época: 5000, loss médio: 0.000000

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.0000)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (1.0000)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (1.0000)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.0000)


Ao alterar o número de épocas para 5000, mantendo a mesma seed, Loss médio diminuiu gradualmente ao longo do tempo até chegar a zero, resultando em respostas exatas nos testes finais.

6.c seed = 80, épocas = 5000, taxa de aprendizado = 0.1

In [44]:
epocas = 5000
taxaAprendizado = 0.1
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.154321
Época: 1000, loss médio: 0.154321
Época: 1500, loss médio: 0.154321
Época: 2000, loss médio: 0.154321
Época: 2500, loss médio: 0.154321
Época: 3000, loss médio: 0.154321
Época: 3500, loss médio: 0.154321
Época: 4000, loss médio: 0.154321
Época: 4500, loss médio: 0.154321
Época: 5000, loss médio: 0.154321

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.5556)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.5556)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (1.0000)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.0000)


Ao aumentar o valor da taxa de aprendizado, o programa dá “saltos” muito grandes no processo da mudança de parâmetros, convergindo para um mínimo local e estagnando a Loss Média.

6.d seed =80, épocas = 10000, taxa de aprendizado = 0.001

In [48]:
epocas = 10000
taxaAprendizado = 0.001
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.207551
Época: 1000, loss médio: 0.175352
Época: 1500, loss médio: 0.163745
Época: 2000, loss médio: 0.160753
Época: 2500, loss médio: 0.159489
Época: 3000, loss médio: 0.158435
Época: 3500, loss médio: 0.157297
Época: 4000, loss médio: 0.156017
Época: 4500, loss médio: 0.154556
Época: 5000, loss médio: 0.152890
Época: 5500, loss médio: 0.150978
Época: 6000, loss médio: 0.148775
Época: 6500, loss médio: 0.146223
Época: 7000, loss médio: 0.143257
Época: 7500, loss médio: 0.139763
Época: 8000, loss médio: 0.135659
Época: 8500, loss médio: 0.130791
Época: 9000, loss médio: 0.125000
Época: 9500, loss médio: 0.118579
Época: 10000, loss médio: 0.114401

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.4863)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.5314)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.9821)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (-0.0127)


7. Avalie trocar a função de ativação ReLU pelas funções de ativação softmax e identidade. Verifique se ha necessidade
de mudar a forma de inicialização dos parâmetros da MLP (se sim, pesquisar por Glorot/Xavier initialization).
Descreva os resultados.

7.a Trocando ReLu por softmax

In [50]:
# transforma o vetor de saídas em probabilidades que somam 1
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # estabilidade numérica
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = softmax(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = softmax(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.258847
Época: 1000, loss médio: 0.254597
Época: 1500, loss médio: 0.247540
Época: 2000, loss médio: 0.243798

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.5347)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.3762)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.6852)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.4076)


Quando usada em uma camada oculta, o Ssoftmax "achata" todos os valores, o que dificulta o aprendizado, pois todas as ativações dependem umas das outras. Como pode ser visto, a rede tende a parar de aprender muito rapidamente e a loss fica quase constante.

7.b Trocando ReLu pela funçãi identidade

In [51]:
def identidade(x):
    return x

epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [0]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = identidade(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = identidade(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.272424
Época: 1000, loss médio: 0.268273
Época: 1500, loss médio: 0.265282
Época: 2000, loss médio: 0.263698

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.5137)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.4976)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.4825)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.4664)


A função identidade não introduz não-linearidade, logo a rede se torna apenas uma grande regressão linear. Portanto, a rede não aprende XOR, e a loss diminui muito pouco.

8. Avalie experimentalmente a aplicação da arquitetura padrão (considerando a função de ativação ReLU) para
aproximar outras funções binárias. Considere, ao menos, AND e OR (ver tabela-verdade abaixo), definidas sobre
numeros binários de forma análoga à função XOR. Descreva os resultados.

8.a aproximação da função AND

In [55]:
epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [0], [0], [1]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.032069
Época: 1000, loss médio: 0.000212
Época: 1500, loss médio: 0.000000
Época: 2000, loss médio: 0.000000

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (-0.0000)
Entrada: [0 1], Saída Esperada: 0, Previsão da Rede: (0.0000)
Entrada: [1 0], Saída Esperada: 0, Previsão da Rede: (0.0000)
Entrada: [1 1], Saída Esperada: 1, Previsão da Rede: (1.0000)


8.b Aproximação da função OR

In [58]:
epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[0], [1], [1], [1]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.043887
Época: 1000, loss médio: 0.020313
Época: 1500, loss médio: 0.006428
Época: 2000, loss médio: 0.001323

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.0124)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (0.9472)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (0.9930)
Entrada: [1 1], Saída Esperada: 1, Previsão da Rede: (1.0412)


8.c aproximação da função XNOR

In [59]:
epocas = 2000
taxaAprendizado = 0.01
entradas = np.array([0, 0]).reshape(1,2)
np.random.seed(80) # seed para os pesos e vieses

#Hiperparâmetros iniciais
pesosIniciais = heInitialization(2, 2)
pesosFinais = heInitialization(2, 1)
viesesIniciais = np.full((1,2),0.1)
viesFinal = np.full((1,1),0.1)
entradasV = np.array([[0,0], [0,1], [1,0], [1,1]])
saidasV = np.array([[1], [0], [0], [1]])

for i in range(epocas):
    loss_total = 0

    for x_entrada, y_saida in zip(entradasV, saidasV):

        entrada = x_entrada.reshape(1,2)
        saidaCorreta = y_saida

        #calcula a saida da camada oculta
        saidaOcultaLinear = entrada @ pesosIniciais
        saidaOcultaLinear = saidaOcultaLinear + viesesIniciais
        saidaOculta = Relu(saidaOcultaLinear)


        #calcula a saida final
        saidaFinal = saidaOculta @ pesosFinais
        saidaFinal = saidaFinal + viesFinal

        #calcula a loss e os gradientes
        loss = calculoLoss(saidaFinal, saidaCorreta)
        loss_total += loss
        gradientes = backWardPass(entrada, saidaCorreta, saidaFinal, pesosFinais, saidaOculta, saidaOcultaLinear)

        #define novos parâmetros
        pesosFinais = calculoParametro(pesosFinais, taxaAprendizado, gradientes[0]) #arrumar
        viesFinal = calculoParametro(viesFinal,taxaAprendizado, gradientes[1])
        pesosIniciais = calculoParametro(pesosIniciais, taxaAprendizado, gradientes[2])
        viesesIniciais = calculoParametro(viesesIniciais, taxaAprendizado, gradientes[3])

    if(i + 1) % 500 == 0:
        loss_medio = loss_total / len(entradasV)
        print(f"Época: {i + 1}, loss médio: {loss_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradasV, saidasV):
    entrada = x_exemplo.reshape(1, 2)
    saidaOcultaLinear = entrada @ pesosIniciais + viesesIniciais
    saidaOculta = Relu(saidaOcultaLinear)
    saidaFinal = saidaOculta @ pesosFinais + viesFinal
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo[0]}, Previsão da Rede: ({saidaFinal[0][0]:.4f})")

Época: 500, loss médio: 0.022581
Época: 1000, loss médio: 0.000007
Época: 1500, loss médio: 0.000000
Época: 2000, loss médio: 0.000000

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 1, Previsão da Rede: (1.0000)
Entrada: [0 1], Saída Esperada: 0, Previsão da Rede: (-0.0000)
Entrada: [1 0], Saída Esperada: 0, Previsão da Rede: (0.0000)
Entrada: [1 1], Saída Esperada: 1, Previsão da Rede: (1.0000)


Avaliando a aplicação da arquitetura padrão para aproximar outras funções binárias, fica evidente a versatilidade e precisão da rede. As funções AND e XNOR tiveram uma queda muito rápida na loss, aproximando-as perdeitamente em 2000 épocas, enquanto a função OR não conseguiu uma aproximação perfeita no mesmo tempo.